# Creating a Simple Horoscope Program
The goal is to generate a birth chart based on user input and then use the sun sign to retrieve the horoscope from online.

## Importing Libraries

In [ ]:
#import main libraries for creating chart
from kerykeion import AstrologicalSubject, Report,KerykeionChartSVG
import os
#import for web scraping
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import cairosvg
import datetime
import textwrap
from IPython.display import display, HTML

In [ ]:
#Create temporary cache variable
os.environ["KERYKEION_CACHE_DIR"] = "/tmp/kerykeion_cache"

## Generating functions for User Inputs 

In [ ]:
#User input for generating birth chart

#Functions for: Name, year, month, day, hour, minutes, city, country

#ask for name, convert to string
def get_name():
    try:
        name = str(input("""What's your name:"""))
    except:
        name = str(input("""Please enter a valid name:"""))
    
    return name

#ask for year, convert it to number
def get_year():
    while True:
        try:
            year = int(input("What year were you born: "))
            if len(str(year)) != 4:
                raise ValueError("Year must be 4 digits")
            if year > datetime.datetime.today().year:
                raise ValueError("Year can't be in the future")  
            return year
        except ValueError as err:
            print(err)
            continue


#ask for month (in the form of a digit), convert it to number
def get_month():
    months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]

    while True:
        try:
            month = input("Enter month (1-12 or name): ")
            
            # Check if month is a name
            if month.lower() in months:
                month = months.index(month.lower()) + 1 

            month = int(month)

            if month < 1 or month > 12:
                raise ValueError("Invalid month number")

            return month

        except ValueError:
            print("Please enter a valid month between 1-12")

#ask for day (in the form of a digit), convert it to number
days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
def get_day(month):
    while True:
        try: 
            day = int(input(f"Enter day (1-{days_in_month[month-1]}): "))
            
            if day < 1 or day > days_in_month[month-1]:
                raise ValueError("Invalid day")
                
            return day
        
        except ValueError:
            print("Please enter a valid day")
     

#ask for hour, convert it to number

def get_hour():

    while True:
        text = input("Enter hour (1-12 AM/PM or 0-23): ")
        
        try:
            # See if valid integer
            hour = int(text) 
            if hour < 0 or hour > 23:
                raise ValueError()
            
        except ValueError:  
            # Try parsing as timestamp
            try:
                dt = datetime.datetime.strptime(text, "%I %p")
                hour = dt.hour
                
            except ValueError:
                # Invalid 
                print("Invalid input, please enter valid hour")
                continue
                
        return hour
    
#ask for minute
def get_minute():

    while True:
        text = input("Enter minutes (0-59): ")
        
        try:
            minute = int(text)
            if minute < 0 or minute > 59: 
                raise ValueError()
            return minute
            
        except ValueError:
            print("Invalid input, please enter valid minutes")

#ask for city, convert to string
def get_city():
    try:
        city = str(input("What City were you born in:"))
    except:
        city = str(input("Please enter a valid city:"))
    
    return city

#ask for country, convert to string
def get_country():
    try:
        country = str(input("What Country were you born in:"))
    except:
        country = str(input("Please enter a valid Country:"))
    
    return country



### Assigning Variables for user Inputs

In [ ]:
name = get_name()
year = get_year()
month = get_month()
day = get_day(month)
hour = get_hour()
minute = get_minute()
city = get_city()
country = get_country()

## Creating the birth chart

In [ ]:
astro = AstrologicalSubject(name, year,month,day,hour,minute,city,country)

### Generating the Report


In [ ]:
report = Report(astro)
report.print_report()

#### More Detailed Information From the Report (Examples)

In [ ]:
#houses
print(astro.tenth_house)
print(astro.fifth_house)
print(astro.twelfth_house)

In [ ]:
#Planets and Moon
print(astro.moon)
print(astro.jupiter)
print(astro.saturn)

In [ ]:
svg_chart = KerykeionChartSVG(astro)
svg_template = svg_chart.makeTemplate()

with open('chart.svg', 'w') as f:
  f.write(svg_template)

### Extracting Horoscope Based on Sun Sign

In [ ]:
#extract just the sign
sun_sign = astro.sun['sign']
print(sun_sign)

In [ ]:
# Map sign to zodiac url slug
sign_to_url = {
  "Ari": "aries",
  "Tau": "taurus",
  "Gem": "gemini",
  "Can": "cancer",
  "Cap": "capricorn",
  "Vir": "virgo",
  "Sco": "scorpio",
  "Sag": "sagittarius",
  "Aqu": "aquarius",
  "Pis": "pisces",
  "Lib": "libra",
  "Leo": "leo"
}

In [ ]:
url_slug = sign_to_url[sun_sign]

In [ ]:
# Construct hororscope url
url = f"https://www.astrology.com/horoscope/daily/{url_slug}.html" 

In [ ]:
#Scrape Page for Horoscope Text
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

# Extract horoscope 
horo_div = soup.find("div", {"id": "content"})

daily_horoscope = horo_div.p.text


# Replace newline characters with spaces
daily_horoscope = daily_horoscope.replace('\n', ' ')

# Format and display the horoscope without scrollbars
formatted_horoscope = ' '.join(textwrap.wrap(daily_horoscope, width=80))
html_code = f"<div>Daily Horoscope for {sun_sign}:</div><div>{formatted_horoscope}</div>"
display(HTML(html_code))